<a href="https://colab.research.google.com/github/redstoneyard/fmri/blob/master/fmri_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#import os
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from itertools import product




#os.chdir(os.path.dirname(os.path.realpath(__file__)))
#print(os.getcwd())

num_of_features = 5
num_of_trees = 200

# correlation-current_flow_betweenness-matrix-0.000000
# correlation-eigenvector-matrix-0.000000

start_time = time.clock()

all_data_set = []
all_label_set = []

graph_file = open('/content/drive/My Drive/data_for_colab/unweighted-correlation-betweenness-matrix-6-0.500000.csv','r')

labels_universe = [0, 1, 2]
label_pots = {}

for line in graph_file:
    line = (line.rstrip()).split(',')
    patient_label = int(line[0][0]) 
    patient_data = []
    if patient_label not in labels_universe:
        pass
    else:
        for i in range(1,len(line)):
            patient_data.append(float(line[i]))
        all_data_set.append(patient_data)
        all_label_set.append(patient_label)
        if patient_label in label_pots.keys():
            label_pots[patient_label].append(len(all_data_set) - 1)
        else:
            label_pots[patient_label] = [len(all_data_set) - 1]

    
all_data_set = np.array(all_data_set)
all_label_set = np.array(all_label_set)


all_data_set_reduced = SelectKBest(k = num_of_features).fit_transform(all_data_set, all_label_set)


'''
sel = VarianceThreshold(threshold = 0.00003)
all_data_set_reduced = sel.fit_transform(all_data_set)
'''

'''
all_data_set_reduced = all_data_set 
'''

result_file = open ('result_file.txt','w')

label_pots_lists = []
label_pots_keys = label_pots.keys()
for x in label_pots_keys:
    label_pots_lists.append(label_pots[x])

prediction_count = {}
for x in labels_universe:
    prediction_count[x] = {}
    for y in labels_universe:
        prediction_count[x][y] = 0

for k in range(48):
    skipped_set = [k]
    #print(skipped_set)
    training_data_set = []
    training_label_set = []
    test_data_set = []
    test_label_set = []
    for patient_ticket in range(0,len(all_data_set_reduced)):
        patient_label = all_label_set[patient_ticket]
        patient_data = all_data_set_reduced[patient_ticket]
        #print(len(patient_data))
        if patient_ticket in skipped_set:
            test_data_set.append(patient_data)
            test_label_set.append(patient_label)
        else:
            training_data_set.append(patient_data)
            training_label_set.append(patient_label)    
    clf = RandomForestClassifier(n_estimators = num_of_trees, criterion = 'entropy')
    clf.fit(training_data_set, training_label_set)
    #print(test_label_set)
    prediction = clf.predict(test_data_set)
    result_file.write(str(test_label_set)+'\n'+str(prediction)+'\n'+'---\n')
    for i in range(0,len(test_label_set)):
        prediction_count[test_label_set[i]][prediction[i]] += 1
        

        
print(time.clock()-start_time)

print(label_pots)

print(label_pots_lists)

test_case_label = {}
test_case_prediction = {}

for i in labels_universe:
    test_case_label[i] = 0
    for j in labels_universe:
        test_case_label[i] += prediction_count[i][j]
    test_case_prediction[i] = 0
    for j in labels_universe:
        test_case_prediction[i] += prediction_count[j][i]

for i, j in product(labels_universe, labels_universe):
    print(i, j, prediction_count[i][j], float(prediction_count[i][j])/float(test_case_label[i]))
    
for i, j in product(labels_universe, labels_universe):
    print(j, i, prediction_count[j][i], float(prediction_count[j][i])/float(test_case_prediction[i]))
    
print('--End--')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
12.61950499999999
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 1: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], 2: [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]}
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]
0 0 6 0.42857142857142855
0 1 6 0.42857142857142855
0 2 2 0.14285714285714285
1 0 6 0.2608695652173913
1 1 16 0.6956521739130435
1 2 1 0.043478260869565216
2 0 2 0.18181818181818182
2 1 3 0.2727272727272727
2 2 6 0.5454545454545454
0 0 6 0.42857142857142855
1 0 6 0.42857142857142855
2 0 2 0.14285714285714285
0 1 6 0.24
1 1 16 0.64
2 1 3 0.12
0 2 2 0.2222222222222222
1 2 1 0.1111111111111111
2 2 6 0.6666666666666666
--End--


In [ ]:
#import os
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import LeaveOneOut
#from sklearn.model_selection import KFold
from itertools import product
from google.colab import drive

drive.mount('/content/drive')



num_of_features = 5
num_of_trees = 200


start_time = time.clock()

all_data_set = []
all_label_set = []

graph_file = open('/content/drive/My Drive/data_for_colab/unweighted-correlation-betweenness-matrix-6-0.500000.csv','r')

labels_universe = [0, 1, 2]
label_pots = {}

for line in graph_file:
    line = (line.rstrip()).split(',')
    patient_label = int(line[0][0]) 
    patient_data = []
    if patient_label not in labels_universe:
        pass
    else:
        for i in range(1,len(line)):
            patient_data.append(float(line[i]))
        all_data_set.append(patient_data)
        all_label_set.append(patient_label)
        if patient_label in label_pots.keys():
            label_pots[patient_label].append(len(all_data_set) - 1)
        else:
            label_pots[patient_label] = [len(all_data_set) - 1]

    
all_data_set = np.array(all_data_set)
all_label_set = np.array(all_label_set)


all_data_set_reduced = SelectKBest(k = num_of_features).fit_transform(all_data_set, all_label_set)




result_file = open ('result_file.txt','w')

label_pots_lists = []
label_pots_keys = label_pots.keys()
for x in label_pots_keys:
    label_pots_lists.append(label_pots[x])

prediction_count = {}
for x in labels_universe:
    prediction_count[x] = {}
    for y in labels_universe:
        prediction_count[x][y] = 0

loo = LeaveOneOut()
# kf = KFold(n_splits = 5)


for train_index, test_index in loo.split(all_data_set_reduced): # or kf.split(all_data_set_reduced)
    training_data_set, test_data_set = all_data_set_reduced[train_index], all_data_set_reduced[test_index]
    training_label_set, test_label_set = all_label_set[train_index], all_label_set[test_index]
    clf = RandomForestClassifier(n_estimators = num_of_trees, criterion = 'entropy')
    clf.fit(training_data_set, training_label_set)
    #print(test_label_set)
    prediction = clf.predict(test_data_set)
    result_file.write(str(test_label_set)+'\n'+str(prediction)+'\n'+'---\n')
    for i in range(0,len(test_label_set)):
        prediction_count[test_label_set[i]][prediction[i]] += 1
        
print(time.clock()-start_time)

print(label_pots)

print(label_pots_lists)

test_case_label = {}
test_case_prediction = {}

for i in labels_universe:
    test_case_label[i] = 0
    for j in labels_universe:
        test_case_label[i] += prediction_count[i][j]
    test_case_prediction[i] = 0
    for j in labels_universe:
        test_case_prediction[i] += prediction_count[j][i]

for i, j in product(labels_universe, labels_universe):
    print(i, j, prediction_count[i][j], float(prediction_count[i][j])/float(test_case_label[i]))
    
for i, j in product(labels_universe, labels_universe):
    print(j, i, prediction_count[j][i], float(prediction_count[j][i])/float(test_case_prediction[i]))
    
print('--End--')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
13.349397999999999
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 1: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], 2: [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]}
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]
0 0 6 0.42857142857142855
0 1 6 0.42857142857142855
0 2 2 